In [103]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, LSTM
from keras.utils import to_categorical
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.utils import to_categorical
from tqdm import tqdm
from sklearn.model_selection import train_test_split


from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
# import PIL
# from PIL import Image
# from keras.preprocessing.image import img_to_array, load_img

In [104]:
train = pd.read_csv('spec_df.csv')

In [105]:
train_image = []
for i in tqdm(range(train.shape[0])):
    img = image.load_img(train['iname'][i], target_size=(28,28,3), grayscale=False)
    img = image.img_to_array(img)
    img = img/255
    train_image.append(img)
X = np.array(train_image)



100%|██████████| 461/461 [00:00<00:00, 605.40it/s]


In [106]:
le = LabelEncoder() 

train['label'] = le.fit_transform(train['label']) 

y=train['label'].values

y = to_categorical(y)


look in the error..dtype is int but the stype of our label is str

The following are totally acceptable in python:

passing a string representation of an integer into int
passing a string representation of a float into float
passing a string representation of an integer into float
passing a float into int
passing an integer into float

But you get a ValueError if you pass a string representation of a float into int, or a string representation of anything but an integer (including empty string). If you do want to pass a string representation of a float to an int, as @katyhuff points out above, you can convert to a float first, then to an integer:

Link: https://stackoverflow.com/questions/1841565/valueerror-invalid-literal-for-int-with-base-10

i think the reason the second one doesnt work is because the label is not representation of an integer or float

u can use other way to categorise the data : try one hot encoding

In [107]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=75, test_size=0.25, shuffle = True)

In [108]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=(28,28,3)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

In [109]:
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [110]:

model.fit(X_train, y_train, epochs=30, validation_data=(X_test, y_test))

Train on 345 samples, validate on 116 samples
Epoch 1/30
345/345 [==============================] - 1s 2ms/step - loss: 1.0328 - accuracy: 0.5768 - val_loss: 0.7866 - val_accuracy: 0.7672
Epoch 2/30
345/345 [==============================] - 1s 2ms/step - loss: 0.8693 - accuracy: 0.6696 - val_loss: 0.7119 - val_accuracy: 0.7672
Epoch 3/30
345/345 [==============================] - 1s 2ms/step - loss: 0.8394 - accuracy: 0.6696 - val_loss: 0.6549 - val_accuracy: 0.7672
Epoch 4/30
345/345 [==============================] - 1s 2ms/step - loss: 0.8199 - accuracy: 0.6754 - val_loss: 0.7149 - val_accuracy: 0.7759
Epoch 5/30
345/345 [==============================] - 1s 2ms/step - loss: 0.7440 - accuracy: 0.6870 - val_loss: 0.5816 - val_accuracy: 0.7845
Epoch 6/30
345/345 [==============================] - 1s 2ms/step - loss: 0.6699 - accuracy: 0.7101 - val_loss: 0.5102 - val_accuracy: 0.7845
Epoch 7/30
345/345 [==============================] - 1s 2ms/step - loss: 0.5994 - accuracy: 0.7710 - 

In [111]:
scores = model.evaluate(X_test, y_test, verbose = 0)
print("Model evaluation accuracy: ", round(scores[1]*100))


Model evaluation accuracy:  92


In [112]:
def predict(path):
    test_image = []

    img = image.load_img(path, target_size=(28,28,3), grayscale=False)
    img = image.img_to_array(img)
    img = img/255
    test_image.append(img)



    test = np.array(test_image)
    prediction = model.predict_classes(test)
    return prediction

In [ ]:
from tkinter import *
 
from PIL import ImageTk, Image 
from tkinter import messagebox
from tkinter import filedialog 
def open_img(): 
 
    x = openfilename() 

    img = Image.open(x) 

    img = img.resize((250, 250), Image.ANTIALIAS) 


    img = ImageTk.PhotoImage(img) 


    panel = Label(root, image = img) 

    panel.image = img 
    panel.grid(row = 2) 
    
    label = predict(x)
    if label == 0:
        label = "Extrasystole"
    elif label == 1:
        label = "Murmur"
    elif label == 2:
        label = "Normal"
    
    
    messagebox.showinfo('Prediction',label)
        
root = Tk() 

root.title("Image Loader") 

root.geometry('550x500')

root.resizable(width = True, height = True) 


btn = Button(root, text ='open image', command = open_img).grid( row = 1, columnspan = 4) 

root.mainloop() 

